### Deflated Sharpe Ratio

거짓 전략 정리로부터의 주요 결론은 $\max_k{\text{SR}_k} \gg E[\max_k{\text{SR}_k}]$이 아니라면 발견된 전략은 거짓 양성이 될 가능성이 높다. 만약 $E[\max_k{\text{SR}_k}]$를 계산할 수 있으며 이 값을 이용해 전략의 성과가 통계적으로 유의하다는 결론을 내고자 가설을 세우는 귀무가설 $H_0 = E[\max_k{\text{SR}_k}]$를 설정할 수 있다. 그다음 축소 샤프 비율 deflated Sharpe ratio (Bailey and López de Prado 2014) 이 다음과 같이 도출될 수 있다.

$$\text{DSR} = Z \left( \frac{\text{SR} - \mathbb{E}\left[\max_k \{\text{SR}_k\} \right]}{\sqrt{1 - \frac{1}{3}\gamma \text{SR} + \frac{4}{T-1} \text{SR}^2}} \right)$$

여기서, 샘플 수 $T$, 다중 테스트들을 조정한 후 $\text{DSR}$은 참 샤프 비율이라는 귀무가설 하에 $\text{SR}$보다 크거나 같은 샤프 비율을 관측할 확률을 의미한다. $\text{DSR}$의 계산은 $E[\max_k{\text{SR}_k}]$의 추정이 필요하고, 이는 $K$와 $V[\text{SR}_k]$의 추정이 필요하다. 이것이 바로 머신러닝이 도움이 되는 곳이며, 다음에 설명한다.

#### 1. Eﬀective Number of Trials

거짓 전략 정리는 테스트 군 family 내에서 독립 시행 수를 아는 것을 요구한다. 그러나 금융 연구가가 독립 시행을 실행하는 것은 흔한 일이 아니다. 더욱 일반적인 상황은 연구자가 여러 전략을 시도하며, 각 전략에 대해 여러 번의 시행이 실행된다. 아마도 한 전략에 관련된 시행은 다른 전략보다 서로 높은 상관관계를 갖는다. 이러한 상관관계 패턴은 블록 상관관계 행렬로 시각화될 수 있다. 예를 들어, 그림 8.3은 근접한 전략 후의 동일한 투자 유니버스 universe에 대해 6,385번 시행 벡터들 수익률 간의 상관관계 행렬의 실제 예에 예제 세부 묘사에 대해서는 로페즈 데 프라도(2019a)를 참고하자. $\text{ONC}$ 알고리즘(Chapter 4)은 4개의 차별화된 전략의 존재를 발견한다. 따라서 이 예에서 $E[K] = 4$로 추정할 것이다. 이는 보수적인 추정치다. 왜냐하면 독립적인 전략의 참 개수 $K$는 낮은 상관관계의 전략의 수보다 작아야만 하기 때문이다.

#### 2. Variance across Trials

Chapter 8에서 로페즈 데 프라도와 루이스(López de Prado and Lewis 2018)를 따른다. 위의 군집화를 끝내면 ONC는 성공적으로 $N$ 전략을 $K$ 그룹으로 분할한다. 이들 각각은 매우 상관관계가 높은 전략으로 구성된다. 더 나아가 이 군집화를 이용해 $N$ 전략을 $K \ll N$ 군집 레벨의 전략으로 줄일 수 있다. 이들 ‘군집 전략’을 작성하면 $V[\text{SR}_k], \quad k=1,\dots,K$에 대한 추정치를 도출할 수 있다.

주어진 군집 $k$에 대해 목적은 총 군집 수익률 시계열 $S_{k,t}$를 형성하는 것이다. 이는 합계를 위한 가중 방법의 선택이 필요하다. 좋은 후보자는 최소 분산 배분이다. 왜냐하면 이는 높은 분산을 가진 개별 시행이 군집 수익률을 지배하는 것을 방지하기 때문이다. 다음 군집 $k$ 내의 전략 집합을 표기하고, $\mathcal{C}_k \subset \mathcal{C}$ 내의 전략에 국한된 공분산 행렬, $r_{i,t}$는 전략 $i \in \mathcal{C}_k$에 대한 수익률 시계열, 그리고 $w_{k,i}$는 전략 $i \in \mathcal{C}_k$에 연관된 가중치. 그러면 다음을 계산할 수 있다.

$$
\{ w_{k,i} \}_{i \in \mathcal{C}_k} = \frac{\Sigma_k^{-1} \mathbf{1}_k}{\mathbf{1}_k^\top \Sigma_k^{-1} \mathbf{1}_k}
$$

$$
S_{k,t} = \sum_{i \in \mathcal{C}_k} w_{k,i} r_{i,t}
$$

여기서 $\mathbf{1}_k$는 $||\mathcal{C}_k||$ 크기의 1들로 이루어진 특성 벡터다. $w_k$를 계산하는 강건한 방법은 로페즈 데 프라도와 루이스(2018)에서 발견할 수 있다. 이제 계산된 군집 수익률 시계열 $S_{k,t}$로 각 $\text{SR}$을 추정한다($\text{SR}_k$). 그러나 이들 $\text{SR}_k$는 이들의 베팅 빈도가 다를 수 있으므로 아직 비교할 수 없다. 이들을 비교할 수 있도록 첫째 각각을 연율화해야만 한다. 따라서 매년의 평균 베팅 수를 계산한다.

$$
\text{Years}_k = \frac{\text{Last Date}_k - \text{First Date}_k}{365.25}
$$

$$
\text{Frequency}_k = \frac{T_k}{\text{Years}_k}
$$

여기서 $T_k$는 $S_{k,t}$의 길이(기간)이고, $\text{First Date}_k$와 $\text{Last Date}_k$는 $S_k$에 대한 각각 첫째와 마지막 거래일이다. 이것으로 연율화된 샤프 비율 $aSR_k$을 다음과 같이 추정한다.

$$
aSR_k = \frac{\mathbb{E}[\{S_{k,t}\}]\sqrt{\text{Frequency}_k}}{\sqrt{\mathbb{V}[\{S_{k,t}\}]}} = SR_k \sqrt{\text{Frequency}_k}
$$

이제 비교할 수 있는 이들 $aSR_k$를 갖고, 다음과 같이 군집화된 시행의 분산을 추정할 수 있다.

$$
\mathbb{E}[\mathbb{V}[\{\text{SR}_k\}]] = \frac{\mathbb{V}[\{aSR_k\}]}{\text{Frequency}_k}
$$

여기서 $\text{Frequency}_k$는 선택한 군집 $k$의 빈도다. 위 식은 (비연율화된) $\text{SR}$의 추정치의 빈도와 일치하고자 군집화된 시행의 추정된 분산을 선택한 전략의 빈도 관점에서 표현한다.